In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import linear_kernel


In [6]:
data = pd.read_csv('C:\project_mlops\datasets\data_final.csv')

In [7]:
data.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id_movie',
       'original_language', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'release_year', 'return', 'cast', 'crew'],
      dtype='object')

In [12]:
data_ml = data[['title','overview',]]

In [19]:
data['features'] = data_ml['overview']
data['features'] = data.apply(lambda x: ' '.join(x.values.astype(str)),axis=1)

In [20]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['features'])

In [21]:
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [22]:
def get_recomendacion(title, knn_model, data, num_recomendatios=5):
    idx = data[data['title'] == title].index[0]
    
    distancia, indices = knn_model.kneighbors(tfidf_matrix[idx], n_neighbors=num_recomendatios+1)
    
    movie_indices = indices.flatten()[1:]
    return data['title'].iloc[movie_indices]

In [24]:
titulo = input('ingrese un titulo')
recomendacion = get_recomendacion(titulo,knn,data)
print(recomendacion)

245                 The Dark Knight Rises
15                        The Dark Knight
99     Batman v Superman: Dawn of Justice
123                         Batman Begins
45                          Suicide Squad
Name: title, dtype: object
